# Integration of SVM and MVO 

This notebook performs a comparison of the integrated MVO-SVM min variance portfolios obtained via the exact MIP solve vs the ADM methods.

The notebook is intended to run on google Colab.


In [2]:
#!sudo python -m pip install gurobipy==10.0.0 uncomment this to use Gurobi 10
#!sudo python -m pip install gurobipy==9.1.2 #Gurobi 9 was used to generate results in the paper
import gurobipy as gp
from gurobipy import GRB
#from google.colab import drive
import math
import time
#drive.mount('/content/gdrive')
#pth = '/content/gdrive/My Drive/Colab Notebooks/SVM MVO/'
pth = '/'
import sys
sys.path.append(pth + 'python/')

# uncomment this if running locally
from python.packages import *
from python.svm_mvo import *
# %run /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/python/packages.py
# %run /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/python/svm_mvo.py

%matplotlib inline

## Import Forecasts
forecasts = pd.read_pickle(pth +'cache/Forecasts.pkl')
## Import Returns
rets = pd.read_pickle(pth + 'cache/RETS.pkl')
## Import feature vectors
wrds_svm = pd.read_pickle(pth + 'cache/WRDS_ASOF.pkl')
## Import Monthly Prices
prices = pd.read_pickle(pth + 'cache/PRICES.pkl')

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-04


## License for Gurobi in Colab

The following code used to set up the Gurobi License on a Colab instance. At the time that the code was initially written, the ability to use a license was alternating between using Gurobi's web licensing service and the standard license file install. The following code attempts to use Gurobi's web license service, but if that fails will resort to using the standard licensing procedure.

The CSV file should contain the fields given from the Gurobi web license manage. If web licensing does not work it should contain the local license key and it will use grbgetkey to license gurobi on colab machine.

Unfortunately, local licensing will not work at the moment because Gurobi requires an active connection to a University network and I am unaware of a way to VPN on the colab machine.

December 2022 Update -- as of gurobi 10 the WLS service should work.

In [3]:
# #read in licence info
# gurobi_licence = pd.read_csv(pth +'cache/gurobi.csv')
# print("Required info for Gurobi:", gurobi_licence.columns)
# try:
#   #web license try to access it via uoft
#   print("Input WLS Access ID")
#   accessid = input()
#   print("Input WLS License ID")
#   licenseid = input()
#   print("Input WLS Secret Key")
#   secret_key = input()
#   #web license try to access it via uoft
#   e = gp.Env(empty=True)
#   #e.setParam('OutputFlag', 0)
#   e.setParam('WLSACCESSID', accessid)
#   e.setParam('LICENSEID', int(licenseid))
#   e.setParam('WLSSECRET', secret_key)
#   e.start()
# except:
#   !chmod 755 /content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey
#   !/content/gdrive/My\ Drive/Colab\ Notebooks/SVM\ MVO/gurobi/grbgetkey {gurobi_licence.LOCAL[0]}
#   e = gp.Env(empty=True)
#   #chmod 755 grbgetkey
#   e.start()


In [4]:
!df -h

'df' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
!cat /proc/cpuinfo

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [6]:
!cat /proc/meminfo

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [53]:
#convenient function
mth = rets.index[25]
mth

sign = lambda a: int((a>0)) - int((a<0))

#preprocessing 
cov_, mean_ = forecasts[mth]
N = len(mean_)
cov = cov_[:N,:N]
mean = mean_[:N,:]
tics = list(rets.columns[:N])
print("valid tickers:", tics)
#get the wharton research data for the valid tickers for the month
wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(mth, level="MonthStart")
#restrict the wharton research data to the columns of interest
Y = wrds_tics.loc[:,"ROC1":"Momentum"] # Y matrix in formulation

Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
Y_ = Y_.iloc[:,:]
cols = ['LEV1', 'GRW1', 'Momentum']
Y_ = Y_[cols]
AssetLim = math.floor(N*0.8)

ret_constr = -1

average_y = (Y_.pow(2).sum(axis = 1)).pow(0.5).mean()
global_epsilon = 0.1*np.sqrt(cov.mean())*average_y
soft_margin = cov.mean()/global_epsilon

#global_epsilon
soft_margin = 0.5
global_epsilon = 0.01

valid tickers: ['AAPL', 'ABC', 'ABMD', 'ABT', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADSK', 'AEE', 'AEP', 'AES', 'AJG', 'AKAM', 'ALB', 'ALK', 'AMAT', 'AMD', 'AME', 'AMGN', 'AMT', 'AMZN', 'ANSS', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ATO', 'AVY', 'AZO', 'BA', 'BAX', 'BBY', 'BDX', 'BKNG', 'BLL', 'BMY', 'BSX', 'BWA', 'CAG', 'CAH', 'CAT', 'CCI', 'CCL', 'CDNS', 'CERN', 'CHD', 'CHRW', 'CI', 'CL', 'CLX', 'CMCSA', 'CMI', 'CMS', 'CNP', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CSCO', 'CSX', 'CTAS', 'CTSH', 'CTXS', 'CVS', 'CVX', 'D', 'DD', 'DGX', 'DHR', 'DIS', 'DISH', 'DLTR', 'DOV', 'DRI', 'DTE', 'DUK', 'DVA', 'DVN', 'DXC', 'EA', 'EBAY', 'ECL', 'ED', 'EFX', 'EIX', 'EL', 'EMN', 'EMR', 'EOG', 'ES', 'ETN', 'ETR', 'EVRG', 'EXC', 'EXPD', 'F', 'FAST', 'FCX', 'FDX', 'FE', 'FFIV', 'FISV', 'FLS', 'FMC', 'GD', 'GIS', 'GLW', 'GPC', 'GPS', 'GWW', 'HAL', 'HAS', 'HD', 'HES', 'HFC', 'HOG', 'HOLX', 'HON', 'HP', 'HPQ', 'HRB', 'HRL', 'HSIC', 'HSY', 'HUM', 'IBM', 'IDXX', 'IEX', 'IFF', 'INCY', 'INTC', 'INTU', 'IP', 'IPG', 'IRM', 

### ADM method with individual updates on the penalty parameters and Random Restarts

### Mean Variance and SVM Joint Using Alternating Directions Method Class Definition with a scalar penalty  parameter

In [54]:

class SVM_MVO_ADM:
    # '''this class models the integrated SVM MVO problem using the ADM solution method'''

    def __init__(self, MVO_, SVM_, IterLim=20, ParamLim=10):
        self.MVO_ = MVO_
        self.SVM_ = SVM_
        self.IterLim = IterLim
        self.ParamLim = ParamLim
        self.x = None
        self.z = None
        self.w = None
        self.b = None
        self.xi_svm = None
        self.xi_mvo = None
        self.track_change = False
        self.change_threshold = 1
    @property
    def describe(self):
        desc = "SVM MVO with Alternating Direction Method"
        shrt = "SVM MVO ADM"
        return desc, shrt

    @property
    def portfolio_risk(self):
        return self.MVO_.x @ self.MVO_.cov @ self.MVO_.x

    @property
    def port_exptd_ret(self):
        return self.MVO_.mean_ret[:, 0] @ self.MVO_.x

    @property
    def soft_penalty(self):
        n, m = self.SVM_.exogenous.shape
        return self.SVM_.soft_penalty

    @property
    def soft_penalty_mvo(self):

        return self.MVO_.soft_penalty

    @property
    def svm_margin(self):
        return (1 / 2) * (self.SVM_.w @ self.SVM_.w)

    @property
    def tics(self):
        return self.MVO_.tics

    @property
    def objective_svm(self):
        objective = self.portfolio_risk.getValue() + self.svm_margin.getValue() + self.soft_penalty.getValue()
        if type(objective) is np.float64:
          return objective
        else:
          return objective[0]

    @property
    def objective_mvo(self):
        objective = self.portfolio_risk.getValue() + self.svm_margin.getValue() + self.soft_penalty_mvo.getValue()
        if type(objective) is np.float64:
          return objective
        else:
          return objective[0]

    def initialize_soln(self, set_return=True, constrs=None, svm_constrs=None, warm_starts=None, delta=0, w_prev_soln=None):
        if svm_constrs is None:
            svm_constrs = []
        if constrs is None:
            constrs = []
        self.MVO_.soft_margin = 0  # on initiliation make sure that the oftmargin is 0
        self.MVO_.set_model(set_return, constrs, warm_starts)  # set up the model
        self.MVO_.optimize()  # find optimal solution
        if self.MVO_.model.status == 4:
            return  # return threshold must be reduced
        self.SVM_.mvo_z = self.MVO_.z.x
        self.SVM_.set_model(svm_constrs, delta, w_prev_soln)
        self.SVM_.optimize()

    def solve_adm(self, store_data=True, set_return=True, constrs=None, svm_constrs=None, delta=0,
                  w_prev_soln=None):
        if svm_constrs is None:
            svm_constrs = []
        if constrs is None:
            constrs = []
        ws = []
        xs = []
        zs = []
        penalty_hist = []
        if self.ParamLim == 1:
            c = np.geomspace(self.SVM_.soft_margin, self.SVM_.soft_margin, self.ParamLim)
        else:
            c = np.geomspace(1e-5, self.SVM_.soft_margin, self.ParamLim)  # initialized to a number > 0
        self.SVM_.soft_margin, self.MVO_.soft_margin = (c[0], c[0])
        xi_mvo = []
        xi_svm = []
        objectives_svm, objectives_mvo = ([], [])
        start = time.time()
        end = time.time()
        z_param_init = self.MVO_.z.x
        x_param_init = self.MVO_.x.x
        for k in range(self.ParamLim):
            self.SVM_.soft_margin, self.MVO_.soft_margin = (c[k], c[k])
            #print(self.SVM_.soft_margin)
            i, converged = (0, False)

            w_param_init = self.SVM_.w.x

            while (i <= self.IterLim) and (not converged):
                w_prev = self.SVM_.w.x
                x_prev = self.MVO_.x.x
                z_prev = self.MVO_.z.x
                if store_data:
                    ws.append(self.SVM_.w.x)
                    xs.append(self.MVO_.x.x)
                    zs.append(self.MVO_.z.x)
                    xi_mvo.append(self.MVO_.xi.x)
                    xi_svm.append(self.SVM_.xi.x)
                    objectives_svm.append(self.objective_svm)
                    objectives_mvo.append(self.objective_mvo)
                    penalty_hist.append(self.SVM_.soft_margin)
                self.MVO_.svm_b = self.SVM_.b.x
                self.MVO_.svm_w = self.SVM_.w.x

                self.MVO_.set_model(set_return, constrs, warm_starts=[x_prev, z_prev])
                self.MVO_.optimize()

                self.SVM_.mvo_z = self.MVO_.z.x
                self.SVM_.set_model(svm_constrs, delta, w_prev_soln)
                self.SVM_.optimize()
                i += 1

                if check_partial_min(self, w_prev):
                    converged = True
                    #print("partial min convergence")
                    if store_data:
                        ws.append(self.SVM_.w.x)
                        xs.append(self.MVO_.x.x)
                        zs.append(self.MVO_.z.x)
                        xi_mvo.append(self.MVO_.xi.x)
                        xi_svm.append(self.SVM_.xi.x)
                        objectives_svm.append(self.objective_svm)
                        objectives_mvo.append(self.objective_mvo)
                        penalty_hist.append(self.SVM_.soft_margin)
                end = time.time()

            if self.track_change:
                if check_global_convergence(self, w_param_init, z_param_init, x_param_init):
                    print("ADM terminated with C = ", np.mean(self.SVM_.soft_margin))
                    break
            else:
                if check_global_convergence(self, w_param_init):
                    print("ADM terminated with C = ", np.mean(self.SVM_.soft_margin))
                    break
            #mult = get_multiplier(self)
            #self.SVM_.soft_margin, self.MVO_.soft_margin = (self.SVM_.soft_margin * mult, self.MVO_.soft_margin * mult)


        self.x = self.MVO_.x
        self.z = self.MVO_.z
        self.w = self.SVM_.w
        self.b = self.SVM_.b
        self.xi_svm = self.SVM_.xi
        self.xi_mvo = self.MVO_.xi
        #self.SVM_.soft_margin, self.MVO_.soft_margin = (
        #    c * (2 ** (self.ParamLim-1)), c * (2 ** (self.ParamLim-1)))  # reinitialize C
        return np.array(ws), np.array(xs), np.array(zs), np.array(xi_mvo), np.array(
            xi_svm), end - start, objectives_svm, objectives_mvo, np.array(penalty_hist)

    def evaluate(self, realized_ret):
        ret = np.dot(self.x.X, realized_ret)
        return ret  # use this to calculate out of sample rets and var

    def get_estimates(self):
        vol_metric = np.sqrt(self.MVO_.portfolio_risk.getValue())[0]
        expt_ret_metric = self.MVO_.port_exptd_ret.getValue()[0]
        return [vol_metric, expt_ret_metric]  # use this for efficient frontiers

    def silence_output(self):
        self.MVO_.model.Params.LogtoConsole = 0
        self.SVM_.model.Params.LogtoConsole = 0

    def get_results(self, export_dir='', fig_size=()):

        lng, shrt = self.describe
        vol_metric = np.sqrt(self.portfolio_risk.getValue())[0]
        expt_ret_metric = self.port_exptd_ret.getValue()[0]
        results = pd.DataFrame(data=np.append(self.x.X, [vol_metric, expt_ret_metric]),
                               index=list(self.tics) + ['Volatility', 'Expected Return'], columns=[lng])

        if export_dir != '':
            results.to_csv(export_dir + 'results.csv')

        if fig_size != () and type(fig_size) in [list, tuple]:
            results[:-2].plot.bar(figsize=fig_size)

        return results.transpose()

    def get_frontier(self, export_dir='', fig_size=(10, 8), lower_ret=None, upper_ret=None):

        n, m = self.SVM_.exogenous.shape
        f = 25
        mean_ret = self.MVO_.mean_ret[:, 0]
        # F is the number of portfolios to use for frontier
        frontier = np.empty((2, f))
        # ws will contain the w's and b values for each portfolio
        ws = np.empty((f, m + 1))
        # xis will contain the w's and b values for each portfolio
        xis = np.empty((f, n))
        # targets for returns
        if lower_ret is None:
            lower_ret = mean_ret.min()
        if upper_ret is None:
            upper_ret = mean_ret.max()
        ret_targ = np.linspace(lower_ret, upper_ret, f)

        for i in range(f):

            constraints = [self.MVO_.port_exptd_ret == ret_targ[i]]
            self.initialize_soln(set_return=False, constrs=constraints)
            self.solve_adm(store_data=False, set_return=False, constrs=constraints)

            if self.MVO_.model.status == 4:
                break

            vol_metric = np.sqrt(self.portfolio_risk.getValue())[0]
            frontier[:, i] = np.array([vol_metric, ret_targ[i]])
            ws[i, :] = np.concatenate([self.w.x, self.b.x])
            xis[i, :] = self.MVO_.xi.x

        if self.MVO_.model.status == 4:
            print("Resolving Model to initial state (return target) then exiting")
            self.MVO_.model.remove(self.MVO_.ret_target)
            self.MVO_.model.update()
            self.MVO_.ret_target = self.MVO_.model.addConstr(self.MVO_.port_exptd_ret >= self.MVO_.ret_constr, 'target')
            self.MVO_.model.optimize()
            return (None, None, None)

        # restore model to original state
        self.MVO_.model.remove(self.MVO_.ret_target)
        self.MVO_.model.update()
        self.MVO_.ret_target = self.MVO_.model.addConstr(self.MVO_.port_exptd_ret >= self.MVO_.ret_constr, 'target')
        self.initialize_soln()
        self.solve_adm()

        fig, ax = plt.subplots(figsize=fig_size)
        # Plot efficient frontier
        ax.plot(frontier[0], frontier[1], '-*', label='Efficient Frontier', color='DarkGreen')

        # Format and display the final plot
        ax.axis([frontier[0].min() * 0.7, frontier[0].max() * 1.3, mean_ret.min() * 1.2, mean_ret.max() * 1.2])
        ax.set_xlabel('Volatility (standard deviation)')
        ax.set_ylabel('Expected Return')
        # ax.legend()
        ax.grid()
        plt.show()
        if (export_dir != ''):
            plt.savefig(export_dir + "EfficientFrontier.png")
        return (frontier, ws, xis)


In [55]:
MVO_ = MVO(tics, mean, cov, ret_constr, Y_, AssetLim, epsilon = global_epsilon)
SVM_ = SVM(tics, Y_ , soft_margin, epsilon = global_epsilon)
SVM_MVO_Fast = SVM_MVO_ADM(MVO_, SVM_, IterLim = 10, ParamLim = 6)
SVM_MVO_Fast.MVO_.model.params.MIPGap = 0.05
SVM_MVO_Fast.SVM_.model.params.timelimit = 60
SVM_MVO_Fast.MVO_.model.params.OutputFlag = 0
SVM_MVO_Fast.SVM_.model.params.OutputFlag = 0
start = time.time()
SVM_MVO_Fast.initialize_soln()
ws , xs, zs , xi_mvo, xi_svm, dt, objs_svm, objs_mvo, penalty_hist = SVM_MVO_Fast.solve_adm()
end = time.time()
print("Solution time", end - start)

Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 60
Solution time 14.557971477508545


In [56]:
SVM_MVO_Fast.svm_margin.getValue() + SVM_MVO_Fast.portfolio_risk.getValue() + SVM_MVO_Fast.soft_penalty.getValue()

array([0.00355629])

In [57]:
MVO_ = MVO(tics, mean, cov, ret_constr, Y_, AssetLim, epsilon = global_epsilon)
SVM_ = SVM(tics, Y_ , soft_margin,epsilon = global_epsilon)
ACS = SVM_MVO_ADM(MVO_, SVM_, IterLim = 10, ParamLim = 1)
ACS.MVO_.model.params.MIPGap = 0.05
ACS.SVM_.model.params.timelimit = 60
ACS.MVO_.model.params.OutputFlag = 0
ACS.SVM_.model.params.OutputFlag = 0
start = time.time()
ACS.initialize_soln()
ws , xs, zs , xi_mvo, xi_svm, dt, objs_svm, objs_mvo, penalty_hist =ACS.solve_adm()
end = time.time()
print("Solution time", end - start)

Set parameter MIPGap to value 0.05
Set parameter TimeLimit to value 60
Solution time 2.671647548675537


In [58]:
ACS.svm_margin.getValue() + ACS.portfolio_risk.getValue() + ACS.soft_penalty.getValue()

array([0.00652832])

In [59]:
ACS.svm_margin.getValue() + ACS.portfolio_risk.getValue() + ACS.soft_penalty.getValue()

array([0.00652832])

## Exact MIP Solution

In [51]:
#SVM with slack 
SVM_MVO_Slck = SVMMVO(tics, mean, cov, ret_constr, soft_margin, Y_, AssetLim, \
                      svm_choice = (True, True), print_var_frntr = False, indicator = False, epsilon = global_epsilon)

In [52]:
SVM_MVO_Slck.model.reset(clearall = 1)
SVM_MVO_Slck.set_model()
SVM_MVO_Slck.model.params.OutputFlag = 0
SVM_MVO_Slck.model.params.timelimit = 100
SVM_MVO_Slck.model.params.MIPGap = 0.05
SVM_MVO_Slck.optimize()

Discarded solution information including additional information


100.02317643165588

In [60]:
SVM_MVO_Slck.model.MIPGap

0.3338624185100719

## Comparison between Exact and ADM method

In [61]:
return_premium = -1 #(Min variance portfolio)
T = len(rets.index)

In [62]:
def evaluate_exact_model(rets, forecasts, wrds_svm, return_premium, model_instance, T, N, cols, link):

  portfolio_weights = np.zeros([T,N])
  xi = np.zeros([T,N])
  z = np.zeros([T,N])

  oot_returns = np.zeros(T)
  risks = np.zeros(T)
  svm_margins = np.zeros(T)
  svm_penalties = np.zeros(T)

  market = np.zeros(T)
  gaps = np.zeros(T)
  M = len(cols)
  wis = np.zeros([T,M])
  bias = np.zeros(T)

  times = np.zeros(T)
  i = 0 #index for dates


  for prd in rets.index.to_list()[:T]:
    ret_ = rets.loc[prd][:N]
    cov_, mean_ = forecasts[prd]
    cov = cov_[:N,:N]
    mean = mean_[:N,:]
    tics = list(rets.columns[:N])
    return_premium_temp = return_premium
    #get the wharton research data for the valid tickers for the month
    wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(prd, level="MonthStart")
    #restrict the wharton research data to the columns of interest
    #modifying WRDS dataset here if required
    Y = wrds_tics.loc[:,cols] # Y matrix in formulation
    Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
    if return_premium == -1:
      ret_constr = -1
    else:
      ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium)
    model_instance.tics = tics
    model_instance.ret_constr = ret_constr
    model_instance.mean_ret = mean
    model_instance.cov = cov
    model_instance.exogenous = Y_

    model_instance.model.reset(clearall = 1)
    model_instance.set_model()


    model_instance.model.Params.LogToConsole = 0
    model_instance.model.setParam('TimeLimit', 100)
    start = time.time()
    model_instance.optimize()
    end = time.time()
    dt = end - start
    print(dt)

    oot_returns[i] = model_instance.evaluate(ret_)
    market[i] = ret_.mean()
    gaps[i] = model_instance.model.MIPGap
    try:
      risks[i] = model_instance.portfolio_risk.getValue()[0]
      svm_margins[i] = model_instance.svm_margin.getValue()[0]
      svm_penalties[i] = model_instance.soft_penalty.getValue()[0]
    except:
      risks[i] = model_instance.portfolio_risk.getValue()
      svm_margins[i] = model_instance.svm_margin.getValue()
      svm_penalties[i] = model_instance.soft_penalty.getValue()

    portfolio_weights[i, :] = model_instance.x.x
    xi[i,:] = model_instance.xi.x
    z[i,:] = model_instance.z.x
    times[i] = dt

    if model_instance.svm_constr:
      wis[i,:] = model_instance.w.x
      bias[i] = model_instance.b.x
    if i + 1 >= T:
      break
    if i%12 == 0:
      print("_"*25)
      print("Iteration ", i)
      print("Percent Complete ", i/T)
      objective_information = pd.DataFrame([oot_returns, market, risks, svm_margins, svm_penalties,times, gaps]
                                        , columns = rets.index[:T],
                                        index = ["Return", "Market", "Risk",
                                                    "Margin", "Infeasibility", "Run-Time", "Gaps"])
      #eligible_weights = pd.DataFrame()
      portfolio_weights_pd = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
      xi_pd = pd.DataFrame(xi, index = rets.index[:T], columns = model_instance.tics)
      z_pd = pd.DataFrame(z, index = rets.index[:T], columns = model_instance.tics)

      svm_weights = pd.DataFrame(wis, index = rets.index[:T], columns = cols)
      svm_weights['bias'] = bias

      svm_results = (portfolio_weights_pd, objective_information.T, svm_weights, z_pd, xi_pd)
      with open(link, 'wb') as fp:
        pkl.dump(svm_results, fp);
    i = i+1
  objective_information = pd.DataFrame([oot_returns, market, risks, svm_margins, svm_penalties,times, gaps]
                                    , columns = rets.index[:T],
                                    index = ["Return", "Market", "Risk",
                                                "Margin", "Infeasibility", "Run-Time", "Gaps"])
  #eligible_weights = pd.DataFrame()
  portfolio_weights_pd = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
  xi_pd = pd.DataFrame(xi, index = rets.index[:T], columns = model_instance.tics)
  z_pd = pd.DataFrame(z, index = rets.index[:T], columns = model_instance.tics)

  svm_weights = pd.DataFrame(wis, index = rets.index[:T], columns = cols)
  svm_weights['bias'] = bias

  svm_results = (portfolio_weights_pd, objective_information.T, svm_weights, z_pd, xi_pd)

  with open(link, 'wb') as fp:
    pkl.dump(svm_results, fp);
  return (portfolio_weights_pd, objective_information.T, svm_weights, z_pd, xi_pd)


In [63]:
link = pth +'cache/Experiments/svm_exact_2023.pkl'
svm_results = evaluate_exact_model(rets, forecasts, wrds_svm, return_premium, SVM_MVO_Slck, T, N, cols, link)

2.1539816856384277
_________________________
Iteration  0
Percent Complete  0.0


In [65]:
def evaluate_adm(rets, forecasts, wrds_svm, return_premium, model_instance, T,
                 N, cols, link):

  portfolio_weights = np.zeros([T,N])
  xi = np.zeros([T,N])
  z = np.zeros([T,N])
  oot_returns = np.zeros(T)
  risks = np.zeros(T)
  svm_margins = np.zeros(T)
  svm_penalties = np.zeros(T)

  market = np.zeros(T)
  M = len(cols)
  wis = np.zeros([T,M])
  bias = np.zeros(T)

  times = np.zeros(T)
  i = 0 #index for dates
  w_mabs = 0 #initialize

  for prd in rets.index.to_list()[:T]:
    ret_ = rets.loc[prd][:N]
    cov_, mean_ = forecasts[prd]
    cov = cov_[:N,:N]
    mean = mean_[:N,:]
    tics = list(rets.columns[:N])
    return_premium_temp = return_premium
    #get the wharton research data for the valid tickers for the month
    wrds_tics = wrds_svm[wrds_svm.index.get_level_values('tic').isin(tics)].xs(prd, level="MonthStart")
    #restrict the wharton research data to the columns of interest
    #modifying WRDS dataset here if required
    Y = wrds_tics.loc[:,cols] # Y matrix in formulation
    Y_ =  (Y - Y.mean(axis=0))/(Y.std(axis=0))
    if return_premium == -1:
      ret_constr = -1
    else:
      ret_constr = mean.mean()*(1 + sign(mean.mean())*return_premium)
    model_instance.MVO_.tics = tics
    model_instance.SVM_.tics = tics
    model_instance.MVO_.ret_constr = ret_constr
    model_instance.MVO_.mean_ret = mean
    model_instance.MVO_.cov = cov
    model_instance.MVO_.exogenous = Y_
    model_instance.SVM_.exogenous = Y_

    model_instance.SVM_.model.reset(clearall = 1)
    model_instance.MVO_.model.reset(clearall = 1)

    start = time.time()
    model_instance.initialize_soln()


    model_instance.silence_output()
    model_instance.solve_adm(store_data=False)
    end = time.time()
    dt = end-start
    print(dt)

    oot_returns[i] = model_instance.evaluate(ret_)
    market[i] = ret_.mean()
    portfolio_weights[i, :] = model_instance.x.x
    xi[i,:] = model_instance.SVM_.xi.x
    z[i,:] = model_instance.z.x
    try:
      risks[i] = model_instance.portfolio_risk.getValue()[0]
      svm_margins[i] = model_instance.svm_margin.getValue()[0]
      svm_penalties[i] = model_instance.soft_penalty.getValue()[0]
    except:
      risks[i] = model_instance.portfolio_risk.getValue()
      svm_margins[i] = model_instance.svm_margin.getValue()
      svm_penalties[i] = model_instance.soft_penalty.getValue()
    portfolio_weights[i, :] = model_instance.x.x
    times[i] = dt
    wis[i,:] = model_instance.w.x
    bias[i] = model_instance.b.x

    if i + 1 >= T:
      break

    if i%12 == 0:
      print("_"*25)
      print("Iteration ", i)
      print("Percent Complete ", i/T)
      objective_information = pd.DataFrame([oot_returns, market, risks, svm_margins, svm_penalties,times]
                                       , columns = rets.index[:T],
                                       index = ["Return", "Market", "Risk",
                                                  "Margin", "Infeasibility", "Run-Time"])
      #eligible_weights = pd.DataFrame()
      portfolio_weights_pd = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
      xi_pd = pd.DataFrame(xi, index = rets.index[:T], columns = model_instance.tics)
      z_pd = pd.DataFrame(z, index = rets.index[:T], columns = model_instance.tics)

      svm_weights = pd.DataFrame(wis, index = rets.index[:T], columns = cols)
      svm_weights['bias'] = bias
      adm_results = (portfolio_weights, objective_information.T, svm_weights, z_pd, xi_pd)
      with open(link, 'wb') as fp:
        pkl.dump(adm_results, fp);
    i = i+1
  objective_information = pd.DataFrame([oot_returns, market, risks, svm_margins, svm_penalties,times]
                                  , columns = rets.index[:T],
                                  index = ["Return", "Market", "Risk",
                                              "Margin", "Infeasibility", "Run-Time"])
  #eligible_weights = pd.DataFrame()
  portfolio_weights_pd = pd.DataFrame(portfolio_weights, index = rets.index[:T], columns = model_instance.tics)
  xi_pd = pd.DataFrame(xi, index = rets.index[:T], columns = model_instance.tics)
  z_pd = pd.DataFrame(z, index = rets.index[:T], columns = model_instance.tics)

  svm_weights = pd.DataFrame(wis, index = rets.index[:T], columns = cols)
  svm_weights['bias'] = bias
  adm_results = (portfolio_weights, objective_information.T, svm_weights, z_pd, xi_pd)
  with open(link, 'wb') as fp:
    pkl.dump(adm_results, fp);
  return (portfolio_weights_pd, objective_information.T, svm_weights, z_pd, xi_pd)

In [66]:
link = pth +'cache/Experiments/svm_padm_2023.pkl'
padm_results = evaluate_adm(rets, forecasts, wrds_svm, return_premium, SVM_MVO_Fast,
                           T, N, cols, link)

21.158535957336426
_________________________
Iteration  0
Percent Complete  0.0
13.086507558822632


In [67]:
link = pth +'cache/Experiments/svm_padm_2023.pkl'
adm_results = evaluate_adm(rets, forecasts, wrds_svm, return_premium, ACS,
                           T, N, cols, link)

6.352819204330444
_________________________
Iteration  0
Percent Complete  0.0
3.8343751430511475
